In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
from pathlib import Path
import time

sys.path.append('src')

from utilities import run_knn, load_ucr_dataset, load_human_action_dataset


c:\Users\Admin\anaconda3\envs\OT\lib\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\OT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
UCR_list = os.listdir("../data/UCR")
Human_Action_list = os.listdir("../data/Human_Actions")
saved_path = "../results"

alg = "FRT"
data = "UCR"  # or "Human_Actions"

In [ ]:
for dataset_name in (Human_Action_list if data == "Human_Actions" else UCR_list):
    if data == "Human_Actions":
        print(f"Processing Human Action dataset: {dataset_name}")
        # Use the Human Action-specific loader for datasets under data/Human_Actions
        X_train, y_train, X_test, y_test = load_human_action_dataset("../data/Human_Actions", dataset_name)
    elif data == "UCR":
        print(f"Processing UCR dataset: {dataset_name}")
        # Use the UCR-specific loader for datasets under data/UCR
        X_train, y_train, X_test, y_test = load_ucr_dataset("../data/UCR", dataset_name)
        
    total_secs = 0
    start = time.time()
    results = run_knn(X_train, y_train, X_test, y_test, alg="FRT")
    total_secs += time.time() - start
    
    out_path = Path(saved_path) / f"knn_{alg}_results_{data}.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)

    cols = ['dataset','acck=1','acck=3','acck=5','acck=10','acck=15','acck=20','build_tree_sec','distance_calc_sec', 'total_secs']
    header_needed = not out_path.exists()

    if isinstance(results, dict):
        keys = [1, 3, 5, 10, 15, 20]
        if alg == "FRT":
            keys += ['build_tree_sec', 'distance_calc_sec']
        vals = [results.get(k, '') for k in keys]
    else:
        vals = list(results)
        if len(vals) < 8:
            vals += [''] * (8 - len(vals))
        vals = vals[:8]

    with out_path.open('a', encoding='utf-8') as f:
        if header_needed:
            f.write(';'.join(cols) + '\n')
        f.write(';'.join([str(dataset_name)] + [str(v) for v in vals] + [str(total_secs)]) + '\n')

Processing UCR dataset: BasicMotions
Successfully loaded dataset: BasicMotions
Size of train data: 40
Size of test data: 40
[FRT] time_weight=88733.543031 | depth_shift=5 | w_avg=9.964542 | points=8000


FRT D_te: 100%|██████████| 40/40 [00:29<00:00,  1.38it/s]


Accuracy (k=1): 0.8000
Accuracy (k=3): 0.7250
Accuracy (k=5): 0.7250
Accuracy (k=10): 0.6250
Accuracy (k=15): 0.5500
Accuracy (k=20): 0.2750
Processing UCR dataset: BME
Successfully loaded dataset: BME
Size of train data: 30
Size of test data: 150
[FRT] time_weight=451.571480 | depth_shift=5 | w_avg=0.100111 | points=23040


FRT D_tr:  40%|████      | 12/30 [00:02<00:02,  6.09it/s]